# Team 11: Predicting the Impact of Twitter Posts on Bitcoin Prices

**Group Members:**  Andrey Bartashevich (14586517), Finn Prins (13458434), Duco Trompert (14591227)

**Research Question:** Can Twitter (a.k.a. X) posts of famous people influence the prices of Bitcoin?

# Hypothesis: 
**Null hypothesis:** There is no impact of Twitter  posts by individuals with over one million followers on the Bitcoin price movements.

**Alternative Hypothesis:** There is an impact of Twitter posts by individuals with over one million followers on the Bitcoin price movements

# Data preprocessing

**Imports**

In [2]:
import pandas as pd
import numpy as np

In [29]:
tweet = pd.read_csv('data/Bitcoin_tweets_dataset_2.csv', engine='python')
tweet['date'] = pd.to_datetime(tweet['date'], errors='coerce')
tweet = tweet.dropna(subset=['date'])
# mill = tweet[tweet['user_followers'] > 1000000]
# print(len(mill))
tweet[['date', 'user_name', 'user_followers', 'text']].head()



,date,user_name,user_followers,text
0,2023-03-01 23:59:59,ChefSam,4680.0,Which #bitcoin books should I think about read...
1,2023-03-01 23:59:47,Roy⚡️,770.0,"@ThankGodForBTC I appreciate the message, but ..."
2,2023-03-01 23:59:42,Ethereum Yoda,576.0,#Ethereum price update: \n\n#ETH $1664.02 USD\...
3,2023-03-01 23:59:36,Viction,236.0,CoinDashboard v3.0 is here\nAvailable on ios a...
4,2023-03-01 23:59:32,Rosie,12731.0,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...


In [30]:
bitcoin = pd.read_csv('data/btcusd_1-min_data.csv')
bitcoin.rename(columns={'Timestamp' : 'date'}, inplace=True)
bitcoin['date'] = pd.to_datetime(bitcoin['date'], unit='s')
bitcoin[['date','Close','Volume']].head()


,date,Close,Volume
0,2012-01-01 10:01:00,4.58,0.0
1,2012-01-01 10:02:00,4.58,0.0
2,2012-01-01 10:03:00,4.58,0.0
3,2012-01-01 10:04:00,4.58,0.0
4,2012-01-01 10:05:00,4.58,0.0


In [32]:
merged = pd.merge(bitcoin[['date', 'Close', 'Volume']], tweet[['date', 'user_name', 'user_followers', 'text']], on='date', how='inner')

print(merged.head())

                 date    Close    Volume          user_name  user_followers  \
0 2023-02-25 20:50:00  22977.0  1.110062             Dani_B           218.0   
1 2023-02-25 20:51:00  23001.0  0.041559       SecretMsgnft           965.0   
2 2023-02-25 20:53:00  22990.0  0.007670  Live Price Crypto          4464.0   
3 2023-02-25 20:53:00  22990.0  0.007670      Manuel Abarca          1272.0   
4 2023-02-25 20:55:00  22990.0  0.010231            운위임에 헨뤼           372.0   

                                                text  
0  Another great series of colour splash\n\nJoin ...  
1  “I’ve developed a new open source P2P e-cash s...  
2  Weekly 🚀\n\n#BTC Last $22921\n\n-RSI: 54\n-Mov...  
3  @xsgames_ #Bitcoin on the Lightning Network\n\...  
4  Stablecoins have been around for a while \nbut...  
